In [7]:
import scraping
import pickle
import create_rawdf
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%autoreload

# データ取得

## 開催日一覧を取得

In [3]:
kaisai_date_list = scraping.scrape_kaisai_date(from_="2024-01", to_="2024-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
race_id_list = scraping.scrape_race_id_list(kaisai_date_list)

  0%|          | 0/106 [00:00<?, ?it/s]

stopped at https://race.netkeiba.com/top/race_list.html?kaisai_date=20240106
Traceback (most recent call last):
  File "/Users/matsukawanaoya/develop/python/keibaAI/common/src/scraping.py", line 77, in scrape_race_id_list
    driver.get(url)
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 454, in get
    self.execute(Command.GET, {"url": url})
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remote/webdriver.py", line 427, in execute
    response = self.command_executor.execute(driver_command, params)
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remote/remote_connection.py", line 404, in execute
    return self._request(command_info[0], url, body=data)
  File "/Users/matsukawanaoya/develop/python/keibaAI/keibavenv/lib/python3.9/site-packages/selenium/webdriver/remote/remote_

レースidリストの読み込みにすごく時間がかかったので<br>
ファイルに保存しておく

In [9]:
# wbはwrite bynariの略
with open("race_id_list.pickle", "wb") as f:
    pickle.dump(race_id_list, f)

## raceページの取得

In [3]:
with open("race_id_list.pickle", "rb") as f:
    race_id_list = pickle.load(f)

In [ ]:
html_paths_race = scraping.scrape_html_race(race_id_list=race_id_list)

In [10]:
# html_paths_raceが失われた場合はファイルから読み込む
if "html_paths_race" not in locals():
    html_paths_race = list(scraping.HTML_RACE_DIR.glob("*.bin"))

In [11]:
results = create_rawdf.create_results(html_path_list=html_paths_race)

0it [00:00, ?it/s]

ValueError: No objects to concatenate

In [6]:
# 欠損チェック
results.isnull().sum()

着順               0
枠番               0
馬番               0
馬名               0
性齢               0
斤量               0
騎手               0
タイム            422
着差            3825
単勝               0
人気             186
馬体重              0
調教師              0
horse_id         0
jockey_id        0
trainer_id       0
owner_id         0
dtype: int64

In [7]:
# 重複チェック
# race_idとhorse_idが同じ行が存在する場合Trueになるのでその件数を確認
results.reset_index()[["race_id", "horse_id"]].duplicated().sum()

np.int64(0)

## horseページからのデータ取得

In [14]:
horse_id_list = results["horse_id"].unique()
len(horse_id_list)

NameError: name 'results' is not defined

In [13]:
html_paths_horse = scraping.scrape_html_horse(
    horse_id_list=horse_id_list, skip=True)

NameError: name 'horse_id_list' is not defined

In [12]:
# 一旦、変数をリセット
del html_paths_horse

In [13]:
# html_paths_horseが失われた場合はファイルから読み込む
if "html_paths_horse" not in locals():
    html_paths_horse = list(scraping.HTML_HORSE_DIR.glob("*.bin"))

In [14]:
# 馬の過去成績テーブルの作成
horse_results = create_rawdf.create_horse_results(html_paths_horse)

  0%|          | 0/11730 [00:00<?, ?it/s]

In [15]:
# horse_results.head()
horse_results.index.nunique()

11730

In [26]:
# レース情報テーブル取得

#htmlを読み込み
if "html_paths_race" not in locals():
    html_paths_race = list(scraping.HTML_RACE_DIR.glob("*.bin"))
# レース情報テーブルの作成
race_info = create_rawdf.create_race_info(html_paths_race)

  0%|          | 0/3454 [00:00<?, ?it/s]

In [ ]:
html_paths_race = list(scraping.HTML_RACE_DIR.glob("*.bin"))
html_paths_race